# LangGraph To-Do App with Subgraph Reminder


#### Subgraph Visualization

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
  +------------+     
  | prioritize |     
  +------------+     
          *          
          *          
          *          
+-----------------+  
| format_reminder |  
+-----------------+  
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      
</pre>

#### Main Graph Visualization

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">    +-----------+      
    | __start__ |      
    +-----------+      
          *            
          *            
          *            
    +----------+       
    | add_todo |       
    +----------+       
          *            
          *            
          *            
+-------------------+  
| reminder_subgraph |  
+-------------------+  
          *            
          *            
          *            
  +---------------+    
  | display_todos |    
  +---------------+    
          *            
          *            
          *            
     +---------+       
     | __end__ |       
     +---------+       
</pre>


## 1. Imports and Setup
- `StateGraph`, `START`, and `END` define the directed workflow used by LangGraph.
- `ChatOllama` and the structured message classes give us an easy way to prompt an LLM.
- `rich.print` keeps runtime tracing readable while we debug or demo the graph.
- If you rely on a different model provider, this is where you would swap in your own client and credentials.


In [ ]:
from typing import List, TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END, START
from rich import print
from dotenv import load_dotenv
import os

# load environment variables from .env file
load_dotenv()  


## 2. Define Shared State
- LangGraph threads a single state dictionary through every node, so we spell it out with a `TodoState` `TypedDict`.
- `todos` uses `Annotated[List[str], operator.add]` so any node can append items across repeated graph runs.
- `new_todo` captures the latest user input, while `reminder` stores whatever message the subgraph produces.


In [ ]:
class TodoState(TypedDict):
    """
    Represents the application's state, including the list of tasks and reminders.
    """
    # Use operator.add to accumulate items into the list across graph runs.
    todos: Annotated[List[str], operator.add]
    reminder: str  # User-friendly reminder message.
    new_todo: str  # The latest todo item to be added.

## 3. Main Graph Nodes
- Main-graph nodes handle user input and output, keeping business logic simple and testable.
- `add_todo` trims the incoming `new_todo`, returns a list of tasks to append, and gracefully ignores blank entries.
- `display_final_state` prints the aggregated tasks plus the reminder, then returns an empty dict because it does not change state.
- Short, focused node functions are easier to unit test and reuse in other graphs.


## 4. Subgraph Nodes
- The reminder workflow lives in a subgraph so we can iterate on it independently of the main app.
- `prioritize_tasks_llm` composes a prompt from the current tasks, calls `ChatOllama`, and handles the edge case where no tasks exist.
- `format_reminder_text` wraps the LLM output in a user-friendly template before handing control back.
- Each node updates only the `reminder` field, illustrating how subgraphs can focus on a narrow slice of shared state.


## 5. Build Graphs
- LangGraph composes nodes by name, so once functions exist we can wire them into reusable workflows.
- Start by declaring the reminder subgraph and then embed it inside the main application graph for a modular design.

### Building Subgraph
- Instantiate `StateGraph(TodoState)` to scope the reminder flow to the shared schema.
- Register the prioritization and formatting nodes, linking `START -> prioritize -> format_reminder -> END`.
- Compile the subgraph to produce `reminder_app`, a callable component you can reuse elsewhere.


### Building Main Graph
- Spin up another `StateGraph(TodoState)` for the top-level workflow.
- Add the `add_todo` node, mount the compiled `reminder_app` as a single step, and finish with `display_todos`.
- Connect the edges in execution order and call `compile()` to create `app`, the final agent we will run multiple times.


In [ ]:

# Create the main application graph.
main_graph = StateGraph(TodoState)

# Add nodes for the main workflow.
main_graph.add_node('add_todo', add_todo)
# Add the compiled subgraph as a single, callable node.
main_graph.add_node('reminder_subgraph', reminder_app)
main_graph.add_node('display_todos', display_final_state)

# Define the data flow for the main graph.
main_graph.add_edge(START, 'add_todo')
main_graph.add_edge('add_todo', 'reminder_subgraph')
main_graph.add_edge('reminder_subgraph', 'display_todos')
main_graph.add_edge('display_todos', END)

# Compile the complete main graph into the final application.
app = main_graph.compile()
print(app.get_graph().draw_ascii())

## 6. Visualize Graph Structure
- Call `draw_ascii()` to confirm the node order without needing extra dependencies.
- Swap in `draw_png()` or `draw_mermaid()` if Graphviz is available and you want richer diagrams in the notebook.
- Visual checks help catch wiring mistakes before you execute the full agent.


## 7. Run the Application
- Begin with an empty state like `{'todos': [], 'reminder': ''}` so the graph can build context over time.
- Invoke `app` multiple times, each call passing the previous state plus a new `new_todo` value to accumulate tasks.
- `operator.add` on the `todos` field preserves prior entries, while the subgraph refreshes the `reminder` each run.
- Inspect `final_state` (or the logged output) to confirm the list and reminder match your expectations.


In [ ]:
# Initialize an empty state for the first run.
current_state = {'todos': [], 'reminder': ''}

# Execute the graph sequentially to add multiple tasks and update the state.
# First invocation adds 'Finish LangGraph tutorial video'.
inputs1 = {'new_todo': 'Finish LangGraph tutorial video'}
# Pass previous state to accumulate tasks.
current_state = app.invoke(inputs1, config={'recursion_limit': 50}, state=current_state)  
print('\nCurrent State:')
print(current_state)

In [ ]:

# Second invocation adds 'Create Basic Grocery List'.
inputs2 = {'new_todo': 'Create Basic Grocery List'}
current_state = app.invoke(inputs2, config={'recursion_limit': 50}, state=current_state)  # Pass previous state.
print('\nCurrent State:')
print(current_state)

In [ ]:
# Third invocation adds 'Schedule dentist appointment' and gets the final result.
inputs3 = {'new_todo': 'Schedule dentist appointment'}
final_state = app.invoke(inputs3, config={'recursion_limit': 50}, state=current_state)  # Pass previous state.

print('\nCurrent State:')
print(current_state)